In [8]:
import tkinter as tk
from tkinter import messagebox
from gensim.summarization.summarizer import summarize
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

def get_text(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return soup.title.text, text

def summarize_text():
    url = entry_url.get()
    word_count_str = entry_word_count.get()

    # check if input values are valid
    if not url or not word_count_str:
        messagebox.showerror("Error", "Please enter both URL and Word Count")
        return
    try:
        word_count = int(word_count_str)
    except ValueError:
        messagebox.showerror("Error", "Word count should be an integer")
        return
    if not is_valid_url(url):
        messagebox.showerror("Error", "Invalid URL. Please enter a valid URL")
        return
    
    messagebox.showinfo("Please Wait", "Summarizing Information in Progress!")
    text = get_text(url)
    number_of_words2 = len(str(text).split())
    summary = summarize(str(text), word_count=word_count)
    lines = summary.split()
    number_of_words = len(lines)
    soup = BeautifulSoup(urlopen(url))
    title = soup.title.get_text()
    result = f"Summary:\n\n{title}\n{summary}\n\nMetrics - \n\nOriginal text character length - {len(str(text))}\nSummarized text character length - {len(summary)}\nOriginal text word count - {number_of_words2}\nSummarized text word count - {number_of_words}"
    text_widget.configure(state='normal')
    text_widget.delete('1.0', tk.END)
    text_widget.insert(tk.END, result)
    text_widget.configure(state='disabled')



def is_valid_url(url):
    regex = re.compile(
        r'^(?:http|ftp)s?://'  # http:// or https:// or ftp:// or ftps://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?))'  # domain...
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, url)    
    
root = tk.Tk()
root.title("Lumos Summarify")
root.geometry("800x650")
root.config(bg="#C8B6A6")

title_label = tk.Label(root, text="Lumos Summarify", font=("Times New Roman", 22, "bold"))
title_label.configure(bg='#C8B6A6')
title_label.pack(fill='both')

frame = tk.Frame(root, bd=2, relief=tk.GROOVE, bg="#C8B6A6")
frame.pack(fill='both', expand=True, padx=10, pady=10)
frame.config(bg="#C8B6A6")

label_url = tk.Label(frame, text="Enter URL: ")
label_url.grid(row=0, column=0, padx=10, pady=10, sticky='w')
label_url.config(bg="#C8B6A6")

entry_url = tk.Entry(frame)
entry_url.grid(row=0, column=1, padx=10, pady=10)

label_word_count = tk.Label(frame, text="Enter word count of summary: ")
label_word_count.grid(row=1, column=0, padx=10, pady=10, sticky='w')
label_word_count.config(bg="#C8B6A6")

entry_word_count = tk.Entry(frame)
entry_word_count.grid(row=1, column=1, padx=10, pady=10)

button = tk.Button(frame, text="Summarize", command=summarize_text)
button.grid(row=2, column=1, pady=10)

text_widget = tk.Text(root, bd=2, relief=tk.GROOVE)
text_widget.pack(fill='both', expand=True, padx=10, pady=10)
text_widget.configure(state='disabled')

def copy_text():
    root.clipboard_clear()
    root.clipboard_append(text_widget.get("1.0", tk.END))
    messagebox.showinfo("Success", "Text copied to clipboard!")

copy_button = tk.Button(root, text="Copy Summary", command=copy_text)
copy_button.pack(side='right', padx=10, pady=10)
copy_button.config(bg='#F1DEC9')

root.mainloop()